In [1]:
from extract_blocks import extract_blocks_from_documents
from ingest import *
from langchain.docstore.document import Document

doc = Document(page_content="Hello, world!", metadata={"author": "Alice"})




In [2]:
class BaseLoader:

    def __init__(self):

        self.data = None

    def load(self):

        return self.data

class LocalDocumentLoader(BaseLoader):

    def __init__(self, file_path):
        super().__init__()
        self.file_path = file_path
        self.data = None


In [2]:
doc.page_content = "\n asdfasdf\n asdfasdf"


In [3]:
document_content_cleaning(doc)
print(doc)

page_content='asdfasdf asdfasdf' metadata={'author': 'Alice'}
